Connected to myenv (Python 3.12.4)

In [4]:
import pandas as pd 
import dash
from dash import html, dcc
from dash.dependencies import Input, Output
import plotly.express as px

spacex_df = pd.read_csv("spacex_launch_geo.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

app = dash.Dash(__name__)
server = app.server
if __name__ == '__main__':
    app.run(debug=False)

uniquelaunchsites = spacex_df['Launch Site'].unique().tolist()
lsites = []
lsites.append({'label': 'All Sites', 'value': 'All Sites'})
for site in uniquelaunchsites:
 lsites.append({'label': site, 'value': site})


app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),


                                dcc.Dropdown(id='site_dropdown',options=lsites,placeholder='Select a Launch Site here', searchable = True , value = 'All Sites'),
                                html.Br(),

                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                # ✅ Second pie chart added here
                                html.Div(dcc.Graph(id='best-site-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                dcc.RangeSlider(
                                    id='payload_slider',
                                    min=0,
                                    max=10000,
                                    step=1000,
                                    marks = {
                                            0: '0 kg',
                                            1000: '1000 kg',
                                            2000: '2000 kg',
                                            3000: '3000 kg',
                                            4000: '4000 kg',
                                            5000: '5000 kg',
                                            6000: '6000 kg',
                                            7000: '7000 kg',
                                            8000: '8000 kg',
                                            9000: '9000 kg',
                                            10000: '10000 kg'
                                    },
                                    value=[min_payload,max_payload]
                                ),
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

@app.callback(
     Output(component_id='success-pie-chart',component_property='figure'),
     [Input(component_id='site_dropdown',component_property='value')]
)
def update_graph(site_dropdown):
    if (site_dropdown == 'All Sites'):
        df  = spacex_df[spacex_df['class'] == 1]
        fig = px.pie(df, names = 'Launch Site',hole=.3,title = 'Total Success Launches By all sites')
    else:
        df  = spacex_df.loc[spacex_df['Launch Site'] == site_dropdown]
        fig = px.pie(df, names = 'class',hole=.3,title = 'Total Success Launches for site '+site_dropdown)
    return fig

@app.callback(
     Output(component_id='success-payload-scatter-chart',component_property='figure'),
     [Input(component_id='site_dropdown',component_property='value'),Input(component_id="payload_slider", component_property="value")]
)
def update_scattergraph(site_dropdown,payload_slider):
    if site_dropdown == 'All Sites':
        low, high = payload_slider
        df  = spacex_df
        mask = (df['Payload Mass (kg)'] > low) & (df['Payload Mass (kg)'] < high)
        fig = px.scatter(
            df[mask], x="Payload Mass (kg)", y="class",
            color="Booster Version",
            size='Payload Mass (kg)',
            hover_data=['Payload Mass (kg)'])
    else:
        low, high = payload_slider
        df  = spacex_df.loc[spacex_df['Launch Site'] == site_dropdown]
        mask = (df['Payload Mass (kg)'] > low) & (df['Payload Mass (kg)'] < high)
        fig = px.scatter(
            df[mask], x="Payload Mass (kg)", y="class",
            color="Booster Version",
            size='Payload Mass (kg)',
            hover_data=['Payload Mass (kg)'])
    return fig

# ✅ New callback for the second pie chart
@app.callback(
    Output(component_id='best-site-pie-chart', component_property='figure'),
    [Input(component_id='site_dropdown', component_property='value')]
)
def update_best_site_pie(site_dropdown):
    # Compute success rates
    site_stats = spacex_df.groupby('Launch Site')['class'].agg(['sum', 'count']).reset_index()
    site_stats['success_rate'] = site_stats['sum'] / site_stats['count']
    best_site = site_stats.sort_values('success_rate', ascending=False).iloc[0]['Launch Site']

    df = spacex_df[spacex_df['Launch Site'] == best_site]
    fig = px.pie(df, names='class', hole=0.3,
                 title=f'Success Rate Breakdown for Best Site: {best_site}')
    return fig

if __name__ == '__main__':
    app.run(debug=False, port=8052)


[2025-05-12 10:57:29,223] ERROR in app: Exception on /_alive_c5e81b6b-3ef5-4ebe-9217-c07dbaf4e639 [GET]
Traceback (most recent call last):
  File "c:\Users\vergamau\OneDrive - HP Inc\01. Pricing & Analytics\Python\myenv\Lib\site-packages\flask\app.py", line 1473, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\vergamau\OneDrive - HP Inc\01. Pricing & Analytics\Python\myenv\Lib\site-packages\flask\app.py", line 882, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\vergamau\OneDrive - HP Inc\01. Pricing & Analytics\Python\myenv\Lib\site-packages\flask\app.py", line 878, in full_dispatch_request
    rv = self.preprocess_request()
         ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\vergamau\OneDrive - HP Inc\01. Pricing & Analytics\Python\myenv\Lib\site-packages\flask\app.py", line 1253, in preprocess_request
    rv = self.ensure_sync(before_func)()
  

In [5]:
import pandas as pd
import dash
from dash import html, dcc
from dash.dependencies import Input, Output
import plotly.express as px

spacex_df = pd.read_csv("spacex_launch_geo.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

app = dash.Dash(__name__)
server = app.server

uniquelaunchsites = spacex_df['Launch Site'].unique().tolist()
lsites = [{'label': 'All Sites', 'value': 'All Sites'}]
for site in uniquelaunchsites:
    lsites.append({'label': site, 'value': site})

app.layout = html.Div(children=[
    html.H1('SpaceX Launch Records Dashboard',
            style={'textAlign': 'center', 'color': '#503D36', 'font-size': 40}),
    
    dcc.Dropdown(id='site_dropdown', options=lsites,
                 placeholder='Select a Launch Site here',
                 searchable=True, value='All Sites'),
    html.Br(),

    html.Div(dcc.Graph(id='success-pie-chart')),
    html.Div(dcc.Graph(id='second-pie-chart')),
    html.Br(),

    html.P("Payload range (Kg):"),
    dcc.RangeSlider(
        id='payload_slider',
        min=0,
        max=10000,
        step=1000,
        marks={i: f'{i} kg' for i in range(0, 11000, 1000)},
        value=[min_payload, max_payload]
    ),
    html.Div(dcc.Graph(id='success-payload-scatter-chart')),
])

@app.callback(
    Output('success-pie-chart', 'figure'),
    Input('site_dropdown', 'value')
)
def update_success_pie(site_dropdown):
    if site_dropdown == 'All Sites':
        df = spacex_df[spacex_df['class'] == 1]
        fig = px.pie(df, names='Launch Site', hole=.3,
                     title='Total Success Launches By all Sites')
    else:
        df = spacex_df[spacex_df['Launch Site'] == site_dropdown]
        fig = px.pie(df, names='class', hole=.3,
                     title=f'Total Success Launches for site {site_dropdown}')
    return fig

@app.callback(
    Output('second-pie-chart', 'figure'),
    Input('site_dropdown', 'value')
)
def update_second_pie(site_dropdown):
    if site_dropdown == 'All Sites':
        df = spacex_df.groupby('Launch Site')['class'].value_counts().unstack(fill_value=0).reset_index()
        df['Total'] = df[0] + df[1]
        fig = px.pie(df, names='Launch Site', values='Total',
                     title='Total Launches (Success + Failure) by Site',
                     hole=0.3)
    else:
        df = spacex_df[spacex_df['Launch Site'] == site_dropdown]
        fig = px.pie(df, names='class', title=f'Success vs Failure for site {site_dropdown}',
                     hole=0.3)
    return fig

@app.callback(
    Output('success-payload-scatter-chart', 'figure'),
    [Input('site_dropdown', 'value'),
     Input('payload_slider', 'value')]
)
def update_scatter(site_dropdown, payload_slider):
    low, high = payload_slider
    if site_dropdown == 'All Sites':
        df = spacex_df
    else:
        df = spacex_df[spacex_df['Launch Site'] == site_dropdown]

    mask = (df['Payload Mass (kg)'] > low) & (df['Payload Mass (kg)'] < high)
    fig = px.scatter(df[mask], x="Payload Mass (kg)", y="class",
                     color="Booster Version", size='Payload Mass (kg)',
                     hover_data=['Payload Mass (kg)'])
    return fig

if __name__ == '__main__':
    app.run(debug=False, port=8052)

In [6]:
import pandas as pd 
import dash
from dash import html, dcc
from dash.dependencies import Input, Output
import plotly.express as px

spacex_df = pd.read_csv("spacex_launch_geo.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

app = dash.Dash(__name__)
server = app.server
if __name__ == '__main__':
    app.run(debug=False)

uniquelaunchsites = spacex_df['Launch Site'].unique().tolist()
lsites = []
lsites.append({'label': 'All Sites', 'value': 'All Sites'})
for site in uniquelaunchsites:
 lsites.append({'label': site, 'value': site})


app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),


                                dcc.Dropdown(id='site_dropdown',options=lsites,placeholder='Select a Launch Site here', searchable = True , value = 'All Sites'),
                                html.Br(),

                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                # ✅ Second pie chart added here
                                html.Div(dcc.Graph(id='best-site-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                dcc.RangeSlider(
                                    id='payload_slider',
                                    min=0,
                                    max=10000,
                                    step=1000,
                                    marks = {
                                            0: '0 kg',
                                            1000: '1000 kg',
                                            2000: '2000 kg',
                                            3000: '3000 kg',
                                            4000: '4000 kg',
                                            5000: '5000 kg',
                                            6000: '6000 kg',
                                            7000: '7000 kg',
                                            8000: '8000 kg',
                                            9000: '9000 kg',
                                            10000: '10000 kg'
                                    },
                                    value=[min_payload,max_payload]
                                ),
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

@app.callback(
     Output(component_id='success-pie-chart',component_property='figure'),
     [Input(component_id='site_dropdown',component_property='value')]
)
def update_graph(site_dropdown):
    if (site_dropdown == 'All Sites'):
        df  = spacex_df[spacex_df['class'] == 1]
        fig = px.pie(df, names = 'Launch Site',hole=.3,title = 'Total Success Launches By all sites')
    else:
        df  = spacex_df.loc[spacex_df['Launch Site'] == site_dropdown]
        fig = px.pie(df, names = 'class',hole=.3,title = 'Total Success Launches for site '+site_dropdown)
    return fig

@app.callback(
     Output(component_id='success-payload-scatter-chart',component_property='figure'),
     [Input(component_id='site_dropdown',component_property='value'),Input(component_id="payload_slider", component_property="value")]
)
def update_scattergraph(site_dropdown,payload_slider):
    if site_dropdown == 'All Sites':
        low, high = payload_slider
        df  = spacex_df
        mask = (df['Payload Mass (kg)'] > low) & (df['Payload Mass (kg)'] < high)
        fig = px.scatter(
            df[mask], x="Payload Mass (kg)", y="class",
            color="Booster Version",
            size='Payload Mass (kg)',
            hover_data=['Payload Mass (kg)'])
    else:
        low, high = payload_slider
        df  = spacex_df.loc[spacex_df['Launch Site'] == site_dropdown]
        mask = (df['Payload Mass (kg)'] > low) & (df['Payload Mass (kg)'] < high)
        fig = px.scatter(
            df[mask], x="Payload Mass (kg)", y="class",
            color="Booster Version",
            size='Payload Mass (kg)',
            hover_data=['Payload Mass (kg)'])
    return fig

# ✅ New callback for the second pie chart
@app.callback(
    Output(component_id='best-site-pie-chart', component_property='figure'),
    [Input(component_id='site_dropdown', component_property='value')]
)
def update_best_site_pie(site_dropdown):
    # Compute success rates
    site_stats = spacex_df.groupby('Launch Site')['class'].agg(['sum', 'count']).reset_index()
    site_stats['success_rate'] = site_stats['sum'] / site_stats['count']
    best_site = site_stats.sort_values('success_rate', ascending=False).iloc[0]['Launch Site']

    df = spacex_df[spacex_df['Launch Site'] == best_site]
    fig = px.pie(df, names='class', hole=0.3,
                 title=f'Success Rate Breakdown for Best Site: {best_site}')
    return fig

if __name__ == '__main__':
    app.run(debug=False, port=8052)

[2025-05-12 10:57:29,873] ERROR in app: Exception on /_alive_c5e81b6b-3ef5-4ebe-9217-c07dbaf4e639 [GET]
Traceback (most recent call last):
  File "c:\Users\vergamau\OneDrive - HP Inc\01. Pricing & Analytics\Python\myenv\Lib\site-packages\flask\app.py", line 1473, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\vergamau\OneDrive - HP Inc\01. Pricing & Analytics\Python\myenv\Lib\site-packages\flask\app.py", line 882, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\vergamau\OneDrive - HP Inc\01. Pricing & Analytics\Python\myenv\Lib\site-packages\flask\app.py", line 878, in full_dispatch_request
    rv = self.preprocess_request()
         ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\vergamau\OneDrive - HP Inc\01. Pricing & Analytics\Python\myenv\Lib\site-packages\flask\app.py", line 1253, in preprocess_request
    rv = self.ensure_sync(before_func)()
  